In [1]:
import glob

In [2]:
import pandas as pd

In [4]:
import xml.etree.ElementTree as ET

In [5]:
from datetime import datetime

In [9]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2021-10-02 11:54:48--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: 'datasource.zip'

     0K ....                                                  100% 2.20G=0s

2021-10-02 11:54:48 (2.20 GB/s) - 'datasource.zip' saved [4249/4249]



In [26]:
import zipfile
with zipfile.ZipFile('datasource1.zip', 'r') as zip_ref:
    zip_ref.extractall('dealership_data')

In [31]:
tmpfile = "dealership_tmp.tmp"

In [32]:
logfile = "dealership_logfile.txt"

In [33]:
targetfile = "dealership_transformed_data.csv"

In [34]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [36]:
def extract_from_json(file_to_process):
    dataframe =pd.read_json(file_to_process,lines=True)
    return dataframe

In [37]:
def extract_from_xml(file_to_process):
    dataframe =pd.DataFrame(columns=['car_model','year_of_manufacture', 'price', 'fuel'])
    tree =ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model= person.find("car_model").text
        year_of_manufacture = int( person.find("year_of_manufacture").text)
        price = float(person.find('price').text)
        fuel = person.find('fuel').text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture": year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
        return dataframe

In [61]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price','fuel'])
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [63]:
extract()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
...,...,...,...,...
58,etios g,2014,6119.402985,Petrol
59,fortuner,2014,29835.820896,Diesel
60,corolla altis,2013,10373.134328,Petrol
61,etios liva,2014,5895.522388,Diesel


In [65]:
def transform(data):
    data['price']=round(data.price,2)
    return data

In [66]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [67]:
def log(message):
    timestamp_format = '%H:%M:%S-%f-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f: f.write(timestamp + ',' + message + 'n')

In [68]:
log("ETL Job Started")

In [69]:
log("Extract phase Started")
extracted_data = extract()

In [74]:
log ("Extract phase Ended")

In [75]:
log("Transform phase Started")

transformed_data = transform(extracted_data)
log("Transform phase Ended")

In [77]:
log("Load phase Started")
load(targetfile, transformed_data)
log("Load phase Ended")

In [78]:
log ("ETL JOb Ended")